<a href="https://colab.research.google.com/github/rismanian1/HW2/blob/main/HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HW2 - Hamed Rismanian 11249700

In [40]:
!pip install sentencepiece
!pip install tensorflow-hub

     |████████████████████████████████| 1.2MB 15.7MB/s 


In [15]:
import tensorflow as tf
import tensorflow_hub as hub
print("TF version: ", tf.__version__)
print("Hub version: ", hub.__version__)

TF version:  2.4.1
Hub version:  0.11.0


In [16]:
import pandas as pd
import numpy as np

# For cleaning the text
import spacy
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import regex as re
import string

# For visualization
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_colwidth', None)

# For building our model
import tensorflow.keras
import sklearn
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dense, Input, Dropout, GlobalAveragePooling1D

### Import the data:

In [ ]:
from google.colab import files
uploaded = files.upload()

In [18]:
import io

TRAIN = pd.read_csv(io.BytesIO(uploaded['train.csv']))

In [24]:
TEST = pd.read_csv(io.BytesIO(uploaded['test.csv']))

In [29]:
TEXT = pd.read_csv(io.BytesIO(uploaded['text.csv']))

In [27]:
NODEID2PAPERID = pd.read_csv(io.BytesIO(uploaded['nodeid2paperid.csv']))

In [ ]:
SAMPLE = pd.read_csv(io.BytesIO(uploaded['sample.csv'])

### Prepare the test set (will be used in final prediction):

In [31]:
test = pd.merge(TEST, NODEID2PAPERID, how='inner', left_on='137832', right_on='node idx') # merging datasets
test = pd.merge(test, TEXT, how='inner', left_on='paper id', right_on='630234') # merging datasets
test = test.drop(columns=['node idx', 'paper id', '630234']) # Remove unnecessary columns
test = test.rename(columns={test.columns[0]: 'id', test.columns[1]: 'title', test.columns[2]: 'abstract'}) # Rename the columns

### Prepare the data (to train models):

In [32]:
data = pd.merge(TRAIN, NODEID2PAPERID, how='inner', left_on='0', right_on='node idx') # merging datasets
data = pd.merge(data, TEXT, how='inner', left_on='paper id', right_on='630234') # merging datasets
data = data.drop(columns=['0', 'node idx', 'paper id', '630234']) # Remove unnecessary columns
data = data.rename(columns={data.columns[0]: 'label', data.columns[1]: 'title', data.columns[2]: 'abstract'}) # Rename the columns

### Explore the data:

In [ ]:
data.info()

In [ ]:
data.head()

In [ ]:
# Length of the titles
data['title'].apply(len).plot(bins=50, kind='hist')

In [ ]:
# Length of the abstracts
data['abstract'].apply(len).plot(bins=50, kind='hist')

In [ ]:
# Bar chart of labels
sns.countplot(x='label', data=data)

### Create a benchmark:

From the graphic above, we see that label 16 has the largest size of all the labels. So as a benchmark, we can predict "16" as the label of each example.

In [ ]:
# Percentage of each category
round(data["label"].value_counts(normalize = True) * 100, 1)

The accuracy of the benchmark model will be approximately 37%.

### Cleaning the Data:

In [36]:
nlp = spacy.load("en")
sp = spacy.load('en_core_web_sm')

nltk.download('stopwords')
nltk.download('punkt')

# spacy (362 words)
spacy_st = nlp.Defaults.stop_words
# nltk(179 words)
nltk_st = stopwords.words('english')

def clean(txt, http = True, punc = True, lem = True, stop_w = True):
    
    if http is True:
        txt = re.sub("https?:\/\/t.co\/[A-Za-z0-9]*", '', txt)

    # stop words
    # in here I changed the placement of lower for those of you who want to use
    # Cased BERT later on.
    if stop_w == 'nltk':
        txt = [word for word in word_tokenize(txt) if not word.lower() in nltk_st]
        txt = ' '.join(txt)

    elif stop_w == 'spacy':
        txt = [word for word in word_tokenize(txt) if not word.lower() in spacy_st]
        txt = ' '.join(txt)

    # lemmitizing
    if lem == True:
        lemmatized = [word.lemma_ for word in sp(txt)]
        txt = ' '.join(lemmatized)

    # punctuation removal
    if punc is True:
        txt = txt.translate(str.maketrans('', '', string.punctuation))
        
    # removing extra space
    txt = re.sub("\s+", ' ', txt)
    
    return txt

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [37]:
data['cleaned_text'] = data.abstract.apply(lambda x: clean(x, lem = True, stop_w = 'nltk', http = True, punc = True))

### Word Embeddings:

In [41]:
# Bert Tokenizer

!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py
import tokenization
FullTokenizer = tokenization.FullTokenizer

In [42]:
ans = input("Which Bert should I use? \n a. Base uncased \n b. Large uncased \n c. Basic cased \n d. Large cased \n")

if ans is 'a':
    BERT_MODEL_HUB = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2'
    disc = 'Base_uncased'
elif ans is 'b':
    BERT_MODEL_HUB = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/2' 
    disc = 'Large_uncased'
elif ans is 'c':
    BERT_MODEL_HUB = 'https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/2'
    disc = 'Base_cased'
elif ans is 'd':
    BERT_MODEL_HUB = 'https://tfhub.dev/tensorflow/bert_en_cased_L-24_H-1024_A-16/2'
    disc = 'Large_cased'

bert_layer = hub.KerasLayer(BERT_MODEL_HUB, trainable=True)
print('Bert layer is ready to use!')



if ans =='a' or ans =='b':
    to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
    vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()

    tokenizer = FullTokenizer(vocabulary_file, to_lower_case)
    

    
elif ans =='c' or ans =='d':
    
    vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
    
    tokenizer = FullTokenizer(vocabulary_file, do_lower_case=False)


print('Bert Tokenizer is ready!!!')

Which Bert should I use? 
 a. Base uncased 
 b. Large uncased 
 c. Basic cased 
 d. Large cased 
a
Bert layer is ready to use!
Bert Tokenizer is ready!!!


In [ ]:
sentence = 'This is the 2nd assignment of Deep Learning.'
print('Tokenized version of {} is : \n {} '.format(sentence, tokenizer.tokenize(sentence)))

In [45]:
def tokenize_text(text_):
    return tokenizer.convert_tokens_to_ids(['[CLS]'] + tokenizer.tokenize(text_) + ['[SEP]'])

data['tokenized_text'] = data.cleaned_text.apply(lambda x: tokenize_text(x))

In [47]:
# the maximum length of the tokenized text

max_len = len(max(data.tokenized_text, key = len))
print('The maximum length is:', max_len)

The maximum length is: 705


In [48]:
# padding
data['padded_text'] = data.tokenized_text.apply(lambda x: x + [0] * (max_len - len(x)))

### Construct the model: